In [2]:
import pandas as pd
import urllib
import requests
from requests.auth import HTTPBasicAuth
import json
import simplejson as json

import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library



In [3]:

where = urllib.parse.quote_plus("""
{
    "County": {
        "$regex": "Riverside"
    },
    "State": "CA"
}
""")
url = 'https://parseapi.back4app.com/classes/US_Zip_Code?&order=Primary_city&where=%s' % where
headers = {
    'X-Parse-Application-Id': 'aE79wHM9EudKOTLWVaIJ1hR0hygU4CAVGvGfetjP', # This is the fake app's application id
    'X-Parse-Master-Key': 'An2S02k3ly1LioZ0AQaKbKsafGnIe8IOrXTooFSn' # This is the fake app's readonly master key
}
data = json.loads(requests.get(url, headers=headers).content.decode('utf-8')) # Here you have the data that you need
# print(json.dumps(data, indent=2))

In [7]:
cities = pd.json_normalize(data['results'])
cities[['CityID','City Name','AKA', 'US_Zip_Code','Latitude','Longitude', 'Pop']] = cities[['objectId', 'Primary_city', 'Acceptable_cities', 'US_Zip_Code', 'Latitude', 'Longitude', 'estimated_population']]

cities[['C_ID', 'C_name', 'C_AKA', 'C_zip', 'C_latitude', 'C_longitude', 'C_population']] = cities[['objectId', 'Primary_city', 'Acceptable_cities', 'US_Zip_Code', 'Latitude', 'Longitude', 'estimated_population']]
df_city = cities[['C_ID', 'C_name', 'C_AKA', 'C_zip', 'C_latitude', 'C_longitude', 'C_population']]
df_city.head(20)

,C_ID,C_name,C_AKA,C_zip,C_latitude,C_longitude,C_population
0,BzRLSPHlJh,Aguanga,NaN,92536,33.50,-116.81,2501
1,KT9ksXMXNi,Anza,NaN,92539,33.53,-116.68,2965
2,7mZMfabAyi,Banning,NaN,92220,33.92,-116.83,21934
3,WrJIF7D6Uk,Beaumont,Cherry Valley,92223,33.94,-116.97,31758
4,WOrUa3rKBA,Blythe,Ripley,92225,33.75,-114.51,10038
5,6S8u4D0eON,Blythe,NaN,92226,33.60,-114.58,1695
6,NrUEf6HtKv,Cabazon,NaN,92230,33.93,-116.77,1532
7,BXq9S7tS59,Calimesa,NaN,92320,33.97,-117.05,6035
8,qYjnV84b0t,Cathedral City,Cathedral Cty,92235,33.77,-116.46,2167
9,EljmvEjFck,Cathedral City,Cathedral Cty,92234,33.82,-116.45,34119


In [8]:
#Combine populations so that we can get the total population of each city
city_pop = df_city.groupby('C_name')['C_population'].sum()
city_pop = city_pop.reset_index()
city_pop.columns = ['C_name', 'Total Pop']
city_pop.shape

(41, 2)

In [9]:
#Remove duplicates city names
df_city = df_city.drop_duplicates(subset=['C_name'])

In [10]:
#merge cities with the df that has the right population count
df_city = pd.merge(df_city, city_pop, on='C_name')
df_city.drop(['C_population'], axis=1, inplace=True)


In [12]:
df_city.head()

,C_ID,C_name,C_AKA,C_zip,C_latitude,C_longitude,Total Pop
0,BzRLSPHlJh,Aguanga,NaN,92536,33.50,-116.81,2501
1,KT9ksXMXNi,Anza,NaN,92539,33.53,-116.68,2965
2,7mZMfabAyi,Banning,NaN,92220,33.92,-116.83,21934
3,WrJIF7D6Uk,Beaumont,Cherry Valley,92223,33.94,-116.97,31758
4,WOrUa3rKBA,Blythe,Ripley,92225,33.75,-114.51,11733


In [13]:
# Keep only top 10 cities
df_cities = df_city.nlargest(20, 'Total Pop')
df_cities.reset_index(drop=True, inplace=True)
df_cities


,C_ID,C_name,C_AKA,C_zip,C_latitude,C_longitude,Total Pop
0,yD8F1K6fJF,Riverside,NaN,92504,33.90,-117.40,295992
1,buIiN7VCGQ,Corona,NaN,92883,33.75,-117.48,175431
2,1OKjEVD32e,Moreno Valley,NaN,92554,33.91,-117.25,135196
3,CW5RzJk9fk,Temecula,NaN,92591,33.53,-117.10,89811
4,yUbWVv0JQC,Murrieta,NaN,92564,33.64,-117.18,88430
5,IrUNNwfYO1,Hemet,NaN,92545,33.73,-117.05,78318
6,QuBC6C8i1T,Perris,NaN,92570,33.78,-117.33,68675
7,O1vc8BPykQ,Indio,NaN,92202,33.66,-116.29,62224
8,vcDofOsMxQ,Lake Elsinore,NaN,92530,33.68,-117.37,46558
9,D3SNPx20FM,Palm Desert,NaN,92211,33.77,-116.34,42823


In [ ]:
# Lets keep only the cities that are in the coachella valley:
# Palm Springs, Palm Desert, Indio, La Quinta, Cathedral City

In [14]:
# find Riverside county lat lng
city = 'Palm Springs, CA'

geolocator = Nominatim(user_agent='riverside_explorer')
location = geolocator.geocode(city)
lat = location.latitude
lng = location.longitude
print('Geo Location is : {} Latitiude, {} Longitude'. format(lat, lng))


Geo Location is : 33.772179449999996 Latitiude, -116.49529769785079 Longitude


In [17]:
# create riverside map
riverside_map = folium.Map(location=[lat,lng], zoom_start=10)
riverside_map


In [16]:
# plot all the cities
for lat, lng, city, pop in zip(df_cities['C_latitude'], df_cities['C_longitude'], df_cities['C_name'], df_cities['Total Pop']):
    label= '{}, {}'.format(city, pop)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        popup=label,
        radius=5,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(riverside_map)

riverside_map

In [ ]:
#empty df of schools
df_schools = pd.DataFrame(columns=['attributes.OBJECTID', 'attributes.NCESID', 'attributes.NAME',
       'attributes.ADDRESS', 'attributes.CITY', 'attributes.STATE',
       'attributes.ZIP', 'attributes.ZIP4', 'attributes.TELEPHONE',
       'attributes.TYPE', 'attributes.STATUS', 'attributes.POPULATION',
       'attributes.COUNTY', 'attributes.COUNTYFIPS', 'attributes.COUNTRY',
       'attributes.LATITUDE', 'attributes.LONGITUDE', 'attributes.NAICS_CODE',
       'attributes.NAICS_DESC', 'attributes.SOURCE', 'attributes.SOURCEDATE',
       'attributes.VAL_METHOD', 'attributes.VAL_DATE', 'attributes.WEBSITE',
       'attributes.LEVEL_', 'attributes.ENROLLMENT', 'attributes.ST_GRADE',
       'attributes.END_GRADE', 'attributes.DISTRICTID',
       'attributes.FT_TEACHER', 'attributes.SHELTER_ID', 'geometry.x',
       'geometry.y'])

# Loop through each city and get the schools and appen to the df_schools database
for city in df_cities['C_name']:
    state = 'CA'
    url_school = "https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Public_Schools/FeatureServer/0/query?where=CITY%20%3D%20'{}'%20AND%20STATE%20%3D%20'{}'&outFields=*&outSR=4326&f=json".format(city, state)
    data_school = json.loads(requests.get(url_school).content.decode('utf-8'))
    schools = pd.json_normalize(data_school['features'])
    df_schools = df_schools.append(schools)
    

Riverside
Corona
Moreno Valley
Temecula
Murrieta
Hemet
Perris
Indio
Lake Elsinore
Palm Desert
Sun City
Cathedral City
Palm Springs
Beaumont
San Jacinto
La Quinta
Coachella
Menifee
Desert Hot Springs
Wildomar


In [ ]:
df_schools.head()

,attributes.OBJECTID,attributes.NCESID,attributes.NAME,attributes.ADDRESS,attributes.CITY,attributes.STATE,attributes.ZIP,attributes.ZIP4,attributes.TELEPHONE,attributes.TYPE,...,attributes.WEBSITE,attributes.LEVEL_,attributes.ENROLLMENT,attributes.ST_GRADE,attributes.END_GRADE,attributes.DISTRICTID,attributes.FT_TEACHER,attributes.SHELTER_ID,geometry.x,geometry.y
0,4484,060243014022,SUMMIT ELEMENTARY,10368 CAMPBELL AVE.,RIVERSIDE,CA,92505,1308,(951) 509-5159,2,...,http://alvordschools.org/,ELEMENTARY,0,KG,05,602430,-999,NOT AVAILABLE,-117.481296,33.935825
1,4485,060243014032,SUMMIT SECONDARY,10368 CAMPBELL AVE.,RIVERSIDE,CA,92505,1308,(951) 509-5159,2,...,http://alvordschools.org/,HIGH,1,06,12,602430,-999,NOT AVAILABLE,-117.481341,33.935549
2,6363,063315014166,EDUCATIONAL OPTIONS CENTER STATE PRESCHOOL,6401 LINCOLN AVE.,RIVERSIDE,CA,92506,4424,(951) 276-7670,1,...,http://www.rusdlink.org/page/484,NOT REPORTED,-999,M,M,633150,-999,NOT AVAILABLE,-117.380239,33.940406
3,6968,063315005007,TOMAS RIVERA ELEMENTARY,20440 RED POPPY LN.,RIVERSIDE,CA,92508,3256,(951) 697-5757,1,...,NOT AVAILABLE,ELEMENTARY,726,KG,06,633150,29,NOT AVAILABLE,-117.306401,33.888458
4,9290,060243000134,LA SIERRA HIGH,4145 LA SIERRA AVE.,RIVERSIDE,CA,92505,2938,(909) 358-1725,1,...,NOT AVAILABLE,HIGH,1842,09,12,602430,80,10798649,-117.479062,33.908754


In [ ]:
for lng, lat, school, city in zip(df_schools['geometry.x'], df_schools['geometry.y'], df_schools['attributes.NAME'], df_schools['attributes.CITY']):
    label= 'School: {}, {}'.format(school, city)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        popup=label,
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(riverside_map)

riverside_map

NameError: name 'df_schools' is not defined

In [ ]:
city = 'Indio'
state = 'CA'
url_school = "https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Public_Schools/FeatureServer/0/query?where=CITY%20%3D%20'{}'%20AND%20STATE%20%3D%20'{}'&outFields=*&outSR=4326&f=json".format(city, state)
data_school = json.loads(requests.get(url_school).content.decode('utf-8'))
# print(json.dumps(data_school, indent = 2))






In [ ]:
schools = pd.json_normalize(data_school['features'])
schools.columns

Index(['attributes.OBJECTID', 'attributes.NCESID', 'attributes.NAME',
       'attributes.ADDRESS', 'attributes.CITY', 'attributes.STATE',
       'attributes.ZIP', 'attributes.ZIP4', 'attributes.TELEPHONE',
       'attributes.TYPE', 'attributes.STATUS', 'attributes.POPULATION',
       'attributes.COUNTY', 'attributes.COUNTYFIPS', 'attributes.COUNTRY',
       'attributes.LATITUDE', 'attributes.LONGITUDE', 'attributes.NAICS_CODE',
       'attributes.NAICS_DESC', 'attributes.SOURCE', 'attributes.SOURCEDATE',
       'attributes.VAL_METHOD', 'attributes.VAL_DATE', 'attributes.WEBSITE',
       'attributes.LEVEL_', 'attributes.ENROLLMENT', 'attributes.ST_GRADE',
       'attributes.END_GRADE', 'attributes.DISTRICTID',
       'attributes.FT_TEACHER', 'attributes.SHELTER_ID', 'geometry.x',
       'geometry.y'],
      dtype='object')